In [1]:
from data_provider.data_creat import *
import akshare as ak

In [5]:
class Args:    
    '''股票数据获取'''
    fuquan = 'hfq'# 设置复权方式,adjust=空选择的不复权，qfq是前复权，应该用hfq后复权来进行量化分析
    period = 'daily' # 拉取时间周期{'daily', 'weekly', 'monthly'}
    start_date = '20151201'  # 20151201 下载数据的开始日期,0就是公司上市时间
    end_date = '-1'  # 下载数据的结束日期,如果0则到最后一天,如果-1是昨天.
    final_data_feat =  ['index', 'Volume','Tom_Chg'] # 删除不需要列的标签
    label_n = 5 # 预测未来连续多少天的收益率
    # 标签是否替换成1或者0
    zhangfu = 0.10  # 预测涨幅大于等于3%的为1，小于3%的为0
    label_ch = True  # 如果是True ，预测n天以后上涨大于变量zhangfu为1，小于为0
    root_path = './dataset/Stock/'
    data_addzero = 60 # 用于训练的数据集分割，前面补0的个数
# 创建参数对象
args = Args()

In [6]:
stock_down = ak.stock_cy_a_spot_em() # 创业板实时数据
stock_list = stock_down[~stock_down['名称'].str.contains("退|ST") & (stock_down['流通市值'] <= 1e11) & (stock_down['总市值'] >= 45e8)] # 去除退市和ST股票
# 保存数据，编码格式为utf-8
file_name = 'Stock_list.csv'
stock_list.to_csv(args.root_path + file_name,index=False,encoding='utf-8-sig')

In [7]:
from sklearn.preprocessing import StandardScaler

all_data_raw = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储原始数据
all_data_scaled = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储标准化后的数据
processed_count = 0  # 初始化计数器
total_count = len(stock_list["代码"].values)  # 获取总股票数量
scaler = StandardScaler()

for i in stock_list["代码"].values[:]:
    NUM = i
    try:
        # 下载原始数据
        raw_data = download_data(NUM, args)
        # 更新已处理股票数量计数器
        processed_count += 1
        # 检查数据长度，如果小于300则跳过此次循环
        if raw_data.shape[0] < 300:
            print(f"股票代码 {NUM} 的数据长度小于300,跳过此次循环。")
            continue
        
        # 拼接数据，添加各种参数
        ad_data = add_data(raw_data.copy(), args)
        # 添加预测标签
        ot_data = add_label(ad_data.copy(), args)
        # 删除无效数据
        final_data_raw = sub_data(ot_data.copy(), args)
        final_data_scaled = final_data_raw.copy()
        
        # 如果args.data_addzero为True，并且all_data_raw不是空的，先将要拼接的数据的前20行替换为0
        if args.data_addzero and not all_data_raw.empty:
            final_data_raw = add_zeros_to_data(final_data_raw, num_rows=args.data_addzero)

        # 存储未经标准化的数据
        all_data_raw = pd.concat([all_data_raw, final_data_raw], ignore_index=True)

        # 标准化处理
        non_time_columns = final_data_scaled.columns[1:-1]  # 假设时间列是第一列
        final_data_scaled[non_time_columns] = scaler.fit_transform(final_data_scaled[non_time_columns])

        # 存储经过标准化的数据
        # 如果args.data_addzero为True，并且all_data_scaled不是空的，先将要拼接的数据的前20行替换为0
        if args.data_addzero and not all_data_scaled.empty:
            final_data_scaled = add_zeros_to_data(final_data_scaled, num_rows=args.data_addzero)

        # 存储经过标准化的数据
        all_data_scaled = pd.concat([all_data_scaled, final_data_scaled], ignore_index=True)

    except Exception as e:
        print(f"处理股票代码 {NUM} 时出现错误: {e}")
        continue

    # 计算并打印处理进度
    progress = processed_count / total_count
    print(f"处理进度: {processed_count}/{total_count} ({progress:.2%})")
    print(f"当前all_data的形状: {all_data_scaled.shape}")

# 保存原始数据到 CSV 文件
file_name_raw = "all_data_raw.csv"
all_data_raw.to_csv(args.root_path + file_name_raw, index=False)

# 保存标准化后的数据到 CSV 文件
file_name_scaled = "all_data_scaled.csv"
all_data_scaled.to_csv(args.root_path + file_name_scaled, index=False)
print("数据保存完毕。")

获取数据时间为： 20151201 - 20231221
原始数据形状： (1883, 9)
添加数据以后形状： (1883, 33)
添加label以后数据形状: (1883, 35)
删除指定行、列后数据形状:  (1853, 33)
处理进度: 1/608 (0.16%)
当前all_data的形状: (1853, 33)
获取数据时间为： 20151201 - 20231221
原始数据形状： (1955, 9)
添加数据以后形状： (1955, 33)
添加label以后数据形状: (1955, 35)
删除指定行、列后数据形状:  (1925, 33)
处理进度: 2/608 (0.33%)
当前all_data的形状: (3778, 33)
数据保存完毕。


In [15]:
# 保存标准化后的数据到 CSV 文件
file_name_scaled = "all_data_scaled.csv"
all_data_scaled.to_csv(args.root_path + file_name_scaled, index=False)
print("数据保存完毕。")

数据保存完毕。
